In [ ]:
%matplotlib inline
from avidares.SingleExperiment import SingleExperiment as SE 
from avidares.FactoredCategoricalHeatmapAnimation import FactoredCategoricalHeatmapAnimation as Heatmap

from IPython.display import HTML
from avidares.BlitArtists import BRectangle, BCellHighlighter, BAnnotation
from avidares.utilities import save_n_html
import matplotlib.pylab as plt
import pdb
import numpy as np

# import sys
# print('example', file=sys.stderr)

SAVEPATH = '/Users/dianeblackwood/_dev/avidaProjects/avidaPeriodicResourceDemo/videos'
#SAVEPATH = '/avidaPeriodicResourceDemo/videos'            #ask Matt about relative path

#Make it an integer if it is one for printing purposes. Not sure how to get the string format to work so I did this.
def makeInt(value=0) :
    rnd = round(value)
    if value == rnd: 
        return int(value)
    else:
        return value


Single Experiments
===============

This notebook demonstrates the avidares utilities to generate and plot a single experiment that uses resources.

There are two main classes: ResourceExperiment and ResourceExperimentAnimation.

ResourceExperiment performs the Avida experiment and loads the resource output file as a Pandas DataFrame.  It receives the following arguments:

    :param environment:  A string representation of the environment file.  Required.
    :param world_size:   A tuple of the (X,Y) size of the world.  Required.
    :param cwd:  The working directory to execute Avida.  Optional.
    :param args:  Arguments to pass to Avida aside from world size and location of input/output files.  Optional.
    :param evnets: The contents of the events file.  If not provided, a default is used. Optional
    :param use_pbar: Show the progress bar

All files created for the experiment are stored in a temporary directory.  This directory and its contents are deleted when the last reference to the ResourceExperiment is removed.

ResourceExperiment.animate() will make an animation of the resource abundances.  It may take up to three arguments:

    :param data_transform:  A function that will transform the resource data in the Pandas DataFrame
    :param figkw:  A dictionary of parameters to pass to the animation initializer method
    :param animkw: A dictionary of parameters to pass to the animation animate() method
    
Meta Data in *.mp4 files
===============
The medat data contains the environment, events and arguments used in the call to avida. 
In a terminal window (on a mac) use the command "ffmpeg -i fineName" to see the meta data. For example

    ffmpeg -i 'Entire Dish for both Inflow(1k) Outflow(0.2) Period(200) Step(1) Updates(700).mp4'

In [ ]:
# xGravity(1) Diffusion(0) Dish(30x30) Inflow=0.1(cyan) Outflow=1(magenta) Period(100) Step(1) Updates(150)
# all runs before 24 Jan may be wrong

# xGravity(1) Diffusion(0) Dish(30x30) Inflow=4(cyan) Outflow=1(magenta) Period(100) Step(1) Updates(750)
# at equilibrium
inColor = 'cyan'
outColor = 'magenta'
inf = 80
outf = 1
inX1 = 0
inY1 = 0
inX2 = 1 
inY2 = 29
outX1 = 28 
outY1 = 0
outX2 = 29 
outY2 = 29
difX = 0 
difY = 0
gravX = 1
gravY = 0
period  =  100
updates =  750
step    =    1
periodK = makeInt(period/1000)
updateK = makeInt(updates/1000)
dishX = 30 
dishY = 30
# The arguments are the string to generate the resource file and the world size as (X,Y)
env = 'RESOURCE res:geometry=grid:inflow=' +str(inf) + ':outflow=' + str(outf)
env += ':inflowx1=' + str(inX1) + ':inflowx2=' +str(inX2) +':inflowy1=' +str(inY1) +':inflowy2=' +str(inY2)
env += ':outflowx1='+str(outX1) + ':outflowx2='+str(outX2)+':outflowy1='+str(outY1)+':outflowy2='+str(outY2)
env += ':xdiffuse=' + str(difX) + ':ydiffuse=' +str(difY) +':xgravity=' +str(gravX)+':ygravity=' +str(gravY)

events ='\
    u begin Inject default-heads.org\n\
    u begin:1:end SetPeriodicResourceInflow res '+str(period)+'\n\
    u 0:' + str(step) + ':end PrintSpatialResources {file_resources}\n\
    u '+str(updates)+' exit\n'

# Setup our expeririment
expr = SE(env, (dishX,dishY), events=events).run()

# Put a box around the inflow
r_in = BRectangle((inX1,inY1), inX2+1-inX1, inY2+1-inY1, ec=str(inColor), lw=1, fc='none')
r_out = BRectangle((outX1,outY1), outX2+1-outX1, outY2+1-outY1, ec=str(outColor), fc='none')

figkw = {
    #'title':'xGravity(1) Dish(30x30) with Inflow=40 (Cyan)\nOutflow=1 (Magenta) Period(100) Udates=550',
    'title':'xGravity('+str(gravX)+') Diffusion('+str(difX)+')'
        + ' Dish('+str(dishX)+'x'+str(dishY)+')'   
        + ' Inflow=' + str(inf) + '(' + str(inColor) + ')'
        + '\nOutflow=' + str(outf) + '(' + str(outColor) + ')' 
        + ' Period(' + str(period) + ')' + ' Step(' + str(step) + ')' + ' Updates(' + str(updates) + ')'
    ,'post_plot':[r_in, r_out]
}

# Run, animate, and display our animation
anim = Heatmap(expr['resources'], expr._world_size, **figkw)
print(anim._vmax)
html = save_n_html(SAVEPATH, anim, expr.config())       #use to print to file
html = anim.animate().to_html5_video()
plt.close()
HTML(html)

In [ ]:
# final demo: X-Diffusion(1) Dish(60x60) Inflow=1 (cyan) at 0 Outflow=1 (magenta) Period(10k) Step(50) Updates(35k)
inColor = 'cyan'
outColor = 'magenta'
inf = 1
outf = 1
inX1 = 0 
inY1 = 0
inX2 = 1 
inY2 = 59
outX1 = 58 
outY1 = 0
outX2 = 59 
outY2 = 59
difX = 1 
difY = 0
period  =  10000
updates =  35000
step    =     50
periodK = makeInt(period/1000)
updateK = makeInt(updates/1000)
dishX = 60 
dishY = 60

# The arguments are the string to generate the resource file and the world size as (X,Y)
env = 'RESOURCE res:geometry=grid:inflow=0.5:outflow=1.0'
env += ':inflowx1=0:inflowy1=0:inflowx2=1:inflowy2=59'
env += ':outflowx1=58:outflowx2=58:outflowy1=0:outflowy2=59'
env += ':xdiffuse=1.0:ydiffuse=0.0'

env = 'RESOURCE res:geometry=grid:inflow=' +str(inf) + ':outflow=' + str(outf)
env += ':inflowx1='+str(inX1) + ':inflowx2=' +str(inX2) +':inflowy1=' +str(inY1) +':inflowy2=' +str(inY2)
env += ':outflowx1='+str(outX1)+':outflowx2='+str(outX2)+':outflowy1='+str(outY1)+':outflowy2='+str(outY2)
env += ':xdiffuse='+str(difX)+':ydiffuse='+str(difY)

events ='\
    u begin Inject default-heads.org\n\
    u begin:1:end SetPeriodicResourceInflow res 40000\n\
    u 0:1000:end PrintSpatialResources {file_resources}\n\
    u 140000 exit\n'
events ='\
    u begin Inject default-heads.org\n\
    u begin:1:end SetPeriodicResourceInflow res '+str(period)+'\n\
    u 0:' + str(step) + ':end PrintSpatialResources {file_resources}\n\
    u '+str(updates)+' exit\n'
# Period(40k) X-Diffusion(1) Dish(60x60) with\nInflow (Cyan) at 0 and Outflow (Magenta) 120k seems enough for equilibrium

# Setup our expeririment
expr = SE(env, (dishX,dishY), events=events).run()

# Put a box around the inflow
r_in = BRectangle((inX1,inY1), inX2+1-inX1, inY2+1-inY1, ec=str(inColor), lw=1, fc='none')
r_out = BRectangle((outX1,outY1), outX2+1-outX1, outY2+1-outY1, ec=str(outColor), fc='none')

figkw = {
#    'title':'X-Diffusion(1) Dish(60x60) Inflow=0.5 (Cyan) at 0\nOutflow=1 (Magenta) Period(40k) Updates=140k',

    'title':'X-Diffusion('+str(difX)+') Dish('+str(dishX)+'x'+str(dishY)+')'  
        + ' Inflow=' + str(inf) + ' (' + str(inColor) + ')' + ' at ' + str(inX1)
        + '\nOutflow=' + str(outf) + ' (' + str(outColor) + ')' 
        + ' Period(' + str(periodK) + 'k)' + ' Step(' + str(step) + ')' + ' Updates(' + str(updateK) + 'k)'
    ,'post_plot':[r_in, r_out]
}

# Run, animate, and display our animation
anim = Heatmap(expr['resources'], expr._world_size, **figkw)
html = save_n_html(SAVEPATH, anim, expr.config())       #use to print to file
# html = anim.animate().to_html5_video()
plt.close()
print(anim._vmax)
print('still a bit slow for a demo')
HTML(html)


In [ ]:
# Final Demo: X-Diffusion(1) Dish(60x60) Inflow=2(cyan) at 25 Outflow=1(magenta) Period(10k) Step(50) Updates(35k)
# 
inColor = 'cyan'
outColor = 'magenta'
inf = 2 
outf = 1
inX1 = 25 
inY1 = 0
inX2 = 26 
inY2 = 59
outX1 = 58 
outY1 = 0
outX2 = 59 
outY2 = 59
difX = 1 
difY = 0

period  =  10000
updates =  35000
step    =     50
periodK = makeInt(period/1000)
updateK = makeInt(updates/1000)
dishX = 60 
dishY = 60

# The arguments are the string to generate the resource file and the world size as (X,Y)
env = 'RESOURCE res:geometry=grid:inflow=2.0:outflow=1.0'
env += ':inflowx1=25:inflowy1=0:inflowx2=26:inflowy2=59'
env += ':outflowx1=58:outflowx2=58:outflowy1=0:outflowy2=59'
env += ':xdiffuse=1.0:ydiffuse=0.0'

env = 'RESOURCE res:geometry=grid:inflow=' +str(inf) + ':outflow=' + str(outf)
env += ':inflowx1='+str(inX1) + ':inflowx2=' +str(inX2) +':inflowy1=' +str(inY1) +':inflowy2=' +str(inY2)
env += ':outflowx1='+str(outX1)+':outflowx2='+str(outX2)+':outflowy1='+str(outY1)+':outflowy2='+str(outY2)
env += ':xdiffuse='+str(difX)+':ydiffuse='+str(difY)

events ='\
    u begin Inject default-heads.org\n\
    u begin:1:end SetPeriodicResourceInflow res '+str(period)+'\n\
    u 0:' + str(step) + ':end PrintSpatialResources {file_resources}\n\
    u '+str(updates)+' exit\n'

# Setup our expeririment
expr = SE(env, (dishX,dishY), events=events).run()

# Put a box around the inflow
r_in = BRectangle((inX1,inY1), inX2+1-inX1, inY2+1-inY1, ec=str(inColor), lw=1, fc='none')
r_out = BRectangle((outX1,outY1), outX2+1-outX1, outY2+1-outY1, ec=str(outColor), fc='none')

figkw = {
    #'title':'X-Diffusion(1) Dish(60x60) Inflow=2 (Cyan) at 25\nand Outflow=1 (Magenta) Period(10k) Updates=495k',
    'title':'X-Diffusion('+str(difX)+') Dish('+str(dishX)+'x'+str(dishY)+')'    
        + ' Inflow=' + str(inf) + '(' + str(inColor) + ')' + ' at ' + str(inX1)
        + '\nOutflow=' + str(outf) + '(' + str(outColor) + ')' 
        + ' Period(' + str(periodK) + 'k)' + ' Step(' + str(step) + ')' + ' Updates(' + str(updateK) + 'k)'
    ,'post_plot':[r_in, r_out]
}

# Run, animate, and display our animation
anim = Heatmap(expr['resources'], expr._world_size, **figkw)
html = save_n_html(SAVEPATH, anim, expr.config())       #use to print to file
html = anim.animate().to_html5_video()
plt.close()
print(anim._vmax)
HTML(html)

In [ ]:
# X-Diffusion(1) Dish(60x60) Inflow=3(cyan) at 40 Outflow=1(magenta) Period(10k) Step(10) Updates(35k);  best of this series

inColor = 'cyan'
outColor = 'magenta'
inf = 3 
outf = 1
inX1 = 40 
inY1 = 0
inX2 = 41 
inY2 = 59
outX1 = 58 
outY1 = 0
outX2 = 59 
outY2 = 59
difX = 1
difY = 0
#period  =  1000
#updates =  3500
#step    =   100
period  =  10000
updates =  35000
step    =     50
periodK = makeInt(period/1000)
updateK = makeInt(updates/1000)
dishX = 60 
dishY = 60
# The arguments are the string to generate the resource file and the world size as (X,Y)
env = 'RESOURCE res:geometry=grid:inflow=4.0:outflow=1.0'
env += ':inflowx1=40:inflowy1=0:inflowx2=41:inflowy2=59'
env += ':outflowx1=58:outflowx2=58:outflowy1=0:outflowy2=59'
env += ':xdiffuse=1.0:ydiffuse=0.0'

env = 'RESOURCE res:geometry=grid:inflow=' +str(inf) + ':outflow=' + str(outf)
env += ':inflowx1='+str(inX1) + ':inflowx2=' +str(inX2) +':inflowy1=' +str(inY1) +':inflowy2=' +str(inY2)
env += ':outflowx1='+str(outX1)+':outflowx2='+str(outX2)+':outflowy1='+str(outY1)+':outflowy2='+str(outY2)
env += ':xdiffuse='+str(difX)+':ydiffuse='+str(difY)

events ='\
    u begin Inject default-heads.org\n\
    u begin:1:end SetPeriodicResourceInflow res '+str(period)+'\n\
    u 0:' + str(step) + ':end PrintSpatialResources {file_resources}\n\
    u '+str(updates)+' exit\n'

# Setup our expeririment
expr = SE(env, (dishX,dishY), events=events).run()

# Put a box around the inflow
r_in = BRectangle((inX1,inY1), inX2+1-inX1, inY2+1-inY1, ec=str(inColor), lw=1, fc='none')
r_out = BRectangle((outX1,outY1), outX2+1-outX1, outY2+1-outY1, ec=str(outColor), fc='none')

figkw = {
    #'title':'X-Diffusion(1) Dish(60x60) Inflow=4 (Cyan) at 40\nOutflow=1 (Magenta) Period(10k) Updates=255k',
    'title':'X-Diffusion('+str(difX)+') Dish('+str(dishX)+'x'+str(dishY)+')'   
        + ' Inflow=' + str(inf) + '(' + str(inColor) + ')' + ' at ' + str(inX1)
        + '\nOutflow=' + str(outf) + '(' + str(outColor) + ')' 
        + ' Period(' + str(periodK) + 'k)' + ' Step(' + str(step) + ')' + ' Updates(' + str(updateK) + 'k)'
    ,'post_plot':[r_in, r_out]
}

# Run, animate, and display our animation
anim = Heatmap(expr['resources'], expr._world_size, **figkw)
html = save_n_html(SAVEPATH, anim, expr.config())       #use to print to file
#html = anim.animate().to_html5_video()
plt.close()
print(anim._vmax)
HTML(html)